# Exercises Meta Heuristics

In [2]:
import math  # type: ignore                                                           # Mathematical functions
import pandas as pd  # type: ignore                                                   # Data manipulation
import numpy as np  # type: ignore                                                    # Scientific computing
import matplotlib.pyplot as plt  # type: ignore                                       # Data visualization
from scipy.stats import binom as binomial  # type: ignore                             # Binomial distribution
from scipy.stats import norm as normal  # type: ignore                                # Normal distribution
from scipy.stats import poisson as poisson  # type: ignore                            # Poisson distribution
from scipy.stats import t as student  # type: ignore                                  # Student distribution
from scipy.stats import chi2  # type: ignore                                          # Chi-squared distribution
from scipy.stats import ttest_1samp  # type: ignore                                   # One-sample t-test
from scipy.stats import chisquare  # type: ignore                                     # Chi-squared test
from scipy.special import comb  # type: ignore                                        # Combinations
from mlxtend.frequent_patterns import apriori  # type: ignore                         # Apriori algorithm
from mlxtend.frequent_patterns import fpgrowth  # type: ignore                        # FP-growth algorithm
from mlxtend.frequent_patterns import association_rules  # type: ignore               # Association rules
from mlxtend.preprocessing import TransactionEncoder  # type: ignore                  # Transaction encoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis  # type: ignore  # Discriminant Analysis
from tensorflow import keras  # type: ignore                                          # Deep Learning library
from tensorflow.keras import Model  # type: ignore                                    # Model class
from tensorflow.keras.layers import Input, Dense, BatchNormalization  # type: ignore  # Layers
from tensorflow.keras.utils import to_categorical  # type: ignore                     # One-hot encoding
from tensorflow. keras.optimizers import Adam  # type: ignore                         # Optimizer
from livelossplot import PlotLossesKeras  # type: ignore                              # Live plot
from keras.src.optimizers import RMSprop  # type: ignore                              # Optimizer
from sklearn.model_selection import train_test_split  # type: ignore                  # Train-test split
from simanneal import Annealer  # type: ignore                                        # Simulated Annealing
import warnings  # type: ignore                                                       # Disable warnings
from Resources.Functions import *  # type: ignore                                     # Custom functions
warnings.filterwarnings("ignore")

# Guidelines
- To solve the problems below, always use simulated annealing and genetic algorithms. So you always provide two solutions. If you have the Tabu search Once implemented, you can even develop a third solution.
- You always have to write a cost function (objective function) yourself that is focused on the problem. You can reuse the cost function for the Simulated Annealing for the genetic algorithms. Note how the objective function should be optimized. You can - if necessary - add a minus sign and/or use rounding.
- Don't forget to also write a function that identifies other solution(s) somewhere in the solution space (e.g. random) or near the current solution(s).
- Experiment with some parameters such as the chance of crossover, or mutation in genetics algorithms.
- In some assignments it is best to also provide lower and upper limits (lower and upper).

## Theoretical questions

### Question 0: Simulated Annealing

In [15]:
class RastriginProblem(Annealer):
    def energy(self):
        x1 = self.state[0]
        x2 = self.state[1]
        sum = 10 * 2 + x1**2 + x2**2
        sum = sum - 10 * math.cos(2 * math.pi * x1) - 10 * math.cos(2 * math.pi * x2)
        return sum # Maximize = return -sum

    def move(self):
        i = np.random.randint(0,2) # Randomly select a dimension (between 0 and 1)
        self.state[i] += np.random.normal(0, 0.1)
        self.state[i] = np.clip(self.state[i], -5.12, 5.12)

init_sol =  np.random.uniform(-5.12,5.12, size=2)
rastrigin = RastriginProblem(init_sol)

rastrigin.Tmax = 25000.0                # Max (starting) temperature
rastrigin.Tmin = 2.5                    # Min (ending) temperature
rastrigin.updates = 10                  # Number of updates (On the screen)
rastrigin.steps = 100000                # Number of iterations
opt_sol = rastrigin.anneal()            # Start the annealing

print(opt_sol)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
    2.50000         11.14    72.28%    36.38%     0:00:01     0:00:00

(array([ 0.00197505, -0.01038512]), 0.02216305119366524)


### Question 0: Traveling Salesman

In [12]:
# Matrix where the distances between the cities are stored (The matrix is symmetric)
distance_matrix = np.array([[0, 100, 125, 100,  75],
                            [100, 0,  50,  75, 100],
                            [125, 50,  0, 100, 125],
                            [100, 75, 100,   0, 50],
                            [75, 100, 125,  50,  0]])

In [13]:
class TravellingSalesmanProblem(Annealer):
    def move(self): # Swaps two cities in the route.
        a = np.random.randint(0, len(self.state) - 1)
        b = np.random.randint(0, len(self.state) - 1)
        self.state[a], self.state[b] = self.state[b], self.state[a]
        
    def energy(self): # Calculates the length of the route.
        dist = 0
        for i in range(len(self.state)):
            dist += distance_matrix[self.state[i - 1], self.state[i]]
        return dist

initial_state = [0, 4, 1, 3, 2]
tsp = TravellingSalesmanProblem(initial_state)
route, distance = tsp.anneal()
route, distance

 Temperature        Energy    Accept   Improve     Elapsed   Remaining


([3, 4, 0, 1, 2], 375)

### Question 1: De rugzak
Je bevindt je in een **geheime kamer** die uitgerust is met een deur met **tijdslot**. Je ziet een timer aftellen die meldt dat je nog maar vijf minuten over het voordat de deur voor altijd op slot zal zijn. Voor je neus liggen **waardevolle voorwerpen** met elk hun eigen **opbrengst en gewicht**. Je hebt een rugzak bij die een absoluut maximaal gewicht kan torsen van 750gr. Op **Canvas** vind je de lijst van voorwerpen met hun gewicht en opbrengst. Stel de optimale rugzak samen. Je zou op een optimale opbrengst van 1458 moeten uitkomen (of toch zeker een waarde dicht daarbij in de buurt).

### Question 2: De dakgoten
Je bent belast met het ontwerp van dakgoten waarbij de productiekost zo laag mogelijk moet zijn. Daarom is het noodzakelijk dat de dakgoten een zo optimale doorsnede hebben met het beschikbare materiaal zodat bladeren en vuil makkelijk afgevoerd kunnen worden. Het bedrijf waarvoor je werkt, koopt **metalen platen** aan die een breedte hebben van `1m. M.a.w. H + B + H` -zie tekening- moet kleiner of gelijk zijn aan `1m`. Bepaal de **ideale `breedte B` en `hoogte H`** van de dakgoot die je uit de platen van `1m` kan maken.

![Gutter](Images/gutter.png)

### Question 3: Het voetbalstadion
De plaatselijke sportclub wil een nieuw stadion bouwen. De omtrek van het sportveld moet `400m` bedragen, en tegelijkertijd willen we ervoor zorgen dat het centrale middenveld een maximale oppervlakte heeft. Bepaal de ideale lengte –en breedteverhouding.

![Football Stadium](Images/footballStadium.png)

### Vraag 4: Optimalisatie

Gegeven volgende te maximaliseren doelfunctie:


$ obj = 0.2 + x_1^2 + x_2^2 - 0.1 \cos(6\pi x_1) - 0.1 \cos(6\pi x_2) $


Met volgende beperkingen: $-1.0 \leq x_i \leq 1.0$ met $i=1,2$

Zoek een goede oplossing.

In [24]:
class XProblem(Annealer):
    def energy(self):
        x = self.state[0]
        y = self.state[1]
        return -(0.2 + x**2 + y**2 - 0.1 * math.cos(6 * math.pi * x) - 0.1 * math.cos(6 * math.pi * y))

    def move(self):
        i = np.random.randint(0,2)
        self.state[i] += np.random.normal(0, 0.1)
        self.state[i] = np.clip(self.state[i], -1, 1)

print(XProblem(np.random.uniform(-1,1, size=2)).anneal())

 Temperature        Energy    Accept   Improve     Elapsed   Remaining


(array([-1.,  1.]), -2.0)


### Question 5: Quiz

1. Hebben bovengemiddelde chromosomen altijd een beter nageslacht? 
2. Kan je met cross-over de hele zoekruimte verkennen? Zijn er beperkingen van deze verkenning? 
3. Waarom speelt de kans op mutatie een belangrijke rol? 
4. Waarom start je bij Simulated Annealing het best met een gerandomiseerde vector?
5. Waarom moet de temperatuur bij simulated annealing afnemen?